In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [3]:
response = wikipedia.invoke('what is the capital of france?')
response

'Page: Closed-ended question\nSummary: A closed-ended question is any question for which a researcher provides research participants with options from which to choose a response. Closed-ended questions are sometimes phrased as a statement that requires a response.\nA closed-ended question contrasts with an open-ended question, which cannot easily be answered with specific information.\n\n\n\nPage: Capital city\nSummary: A capital city or just capital is the municipality holding primary status in a country, state, province, department, or other subnational division, usually as its seat of the government. A capital is typically a city that physically encompasses the government\'s offices and meeting places; the status as capital is often designated by its law or constitution. In some jurisdictions, including several countries, different branches of government are in different settlements, sometimes meaning multiple official capitals. In some cases, a distinction is made between the offic

# PUBMED

In [4]:
from langchain_community.tools import PubmedQueryRun

In [5]:
search = PubmedQueryRun()
print(search.invoke('COVID-19?'))

Published: 2024-12-02
Title: Building on the COVID-19 in Pregnancy in Scotland study to support ongoing surveillance, research, and pandemic preparedness for maternal and child health.
Copyright Information: 
Summary::
No abstract available

Published: 2024-12-02
Title: Post-COVID Mental Health Crises: Globally Minded for Solutions and Solidarity.
Copyright Information: 
Summary::
en masse.
Mental health is deteriorating quickly and significantly globally post-COVID. Though there were already over 1 billion people living with mental disorders pre-pandemic, in the first year of COVID-19 alone, the prevalence of anxiety and depression soared by 25% worldwide. In light of the chronic shortages of mental health provider and resources, along with disruptions of available health services caused by the pandemic and COVID-related restrictions, technology is widely believed to hold the key to addressing rising mental health crises. However, hurdles such as fragmented and often suboptimal patien

# Tool CAlling

In [6]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

@tool
def wikipedia_search(query):
    """
    Search wikipedia for general information.

    Args:
    Query: the search query
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    response = wikipedia.invoke(query)
    return response


@tool
def pubmed_search(query):
    """
    Search pubmed for medical and science research information.

    Args:
    Query: the search query    
    """    
    pubmed = PubmedQueryRun()
    response = pubmed.invoke(query)
    return response

@tool
def duckduckgo_search(query):
    """
    Search duckduckgo for realtime news information.

    Args:
    Query: the search query        
    """    
    wrapper = DuckDuckGoSearchAPIWrapper(region="pt-br", time="d", max_results=2)

    search = DuckDuckGoSearchResults(
            max_results=5,
            # output_format="list",
            api_wrapper=wrapper,
            source="news"
        )

    response = search.invoke(query)
    response

In [7]:
tools = [wikipedia_search, pubmed_search, duckduckgo_search]
list_of_tools = {tool.name: tool for tool in tools}
list_of_tools

{'wikipedia_search': StructuredTool(name='wikipedia_search', description='Search wikipedia for general information.\n\nArgs:\nQuery: the search query', args_schema=<class 'langchain_core.utils.pydantic.wikipedia_search'>, func=<function wikipedia_search at 0x000002306B9A7C70>),
 'pubmed_search': StructuredTool(name='pubmed_search', description='Search pubmed for medical and science research information.\n\nArgs:\nQuery: the search query', args_schema=<class 'langchain_core.utils.pydantic.pubmed_search'>, func=<function pubmed_search at 0x000002306BA084C0>),
 'duckduckgo_search': StructuredTool(name='duckduckgo_search', description='Search duckduckgo for realtime news information.\n\nArgs:\nQuery: the search query', args_schema=<class 'langchain_core.utils.pydantic.duckduckgo_search'>, func=<function duckduckgo_search at 0x000002306BA083A0>)}

In [8]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')

In [36]:
llm_with_tools = llm.bind_tools(tools)

query = "who was the champion of libertadores da américa championship?"
# query = "who is Elon Musk?"

response = llm_with_tools.invoke(query)

In [37]:
response

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-12-02T16:51:17.6431564Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'wikipedia_search', 'arguments': {'query': 'Libertadores da América championship winner'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 713880500, 'load_duration': 35647400, 'prompt_eval_count': 287, 'prompt_eval_duration': 206913000, 'eval_count': 29, 'eval_duration': 469314000}, id='run-214989f6-9a6f-4de1-9765-419503402ab6-0', tool_calls=[{'name': 'wikipedia_search', 'args': {'query': 'Libertadores da América championship winner'}, 'id': 'a98ac4d5-b78e-4542-bac9-ba1ec576425f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 287, 'output_tokens': 29, 'total_tokens': 316})

In [38]:
response.tool_calls

[{'name': 'wikipedia_search',
  'args': {'query': 'Libertadores da América championship winner'},
  'id': 'a98ac4d5-b78e-4542-bac9-ba1ec576425f',
  'type': 'tool_call'}]

In [39]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(query)]

response = llm_with_tools.invoke(messages)
response.tool_calls

[{'name': 'wikipedia_search',
  'args': {'query': 'Libertadores da América champion'},
  'id': '8ce4b96d-306b-44b5-8708-6e6c789d7ddb',
  'type': 'tool_call'}]

In [40]:
import textwrap

answers = []
for tool_call in response.tool_calls:
    name = tool_call['name'].lower()
    selected_tool = list_of_tools[name]
    tool_msg = selected_tool.invoke(tool_call)
    answers.append(tool_msg)

In [41]:
for a in answers:
    print(a)
    # print(textwrap.fill(a.content))

content='Page: Copa Libertadores\nSummary: The CONMEBOL Libertadores, also known as Copa Libertadores de América (Portuguese: Copa/Taça Libertadores da América), is an annual continental club football competition organized by CONMEBOL since 1960. It is the highest level of competition in South American club football. The tournament is named after the Libertadores (Spanish and Portuguese for liberators), the leaders of the Latin American wars of independence, so a literal translation of its former name into English is "Liberators of the Americas Cup".\nThe competition has had several formats over its lifetime. Initially, only the champions of the South American leagues participated. In 1966, the runners-up of the South American leagues began to join. In 1998, Mexican teams were invited to compete and contested regularly from 2000 until 2016. In 2000 the tournament was expanded from 20 to 32 teams. Today at least four clubs per country compete in the tournament, with Argentina and Brazil